In [23]:
import csv
from sklearn.model_selection import train_test_split

In [24]:
def readData(filename):
    with open(filename, 'r') as read_obj:
        csv_reader = csv.reader(read_obj) # Return a reader object which will
                                        # iterate over lines in the given csvfile
        training_data = list(csv_reader)
    return training_data

In [25]:
def convert_to_float(data_set):  
    """ convers the class attribute
    values to float"""


In [26]:
#### Main ####
actual_data_set = readData('wine.csv')
